In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-headlines-for-sarcasm-detection/Data.csv


In [6]:
# Import some of the most important libary for using this notebook
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [7]:
file_path = '/kaggle/input/news-headlines-for-sarcasm-detection/Data.csv'
# Read csv file for using pandas libary
sarcasm_df = pd.read_csv(file_path)
# Looked first five rows
sarcasm_df.head()

,headlines,target
0,CNN Triumphs (At Least in Most Demographic Cat...,Non Sarcastic
1,"‘You Did The Best You Could,’ Says Iron Man Ac...",Sarcastic
2,New Emails Reveal Warm Relationship Between Ka...,Non Sarcastic
3,Donald Trump Jr. Gets Slammed Over Racist Birt...,Non Sarcastic
4,God Urges Rick Perry Not To Run For President,Sarcastic


In [8]:
# Split them two column
input_seq = sarcasm_df['headlines']
target_seq = sarcasm_df['target']

len(input_seq),len(target_seq)

(12506, 12506)

In [9]:
vocab_size = 10000 # It's most common vocabulary size.
embedding_dim = 16 
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [10]:
input_seq

0        CNN Triumphs (At Least in Most Demographic Cat...
1        ‘You Did The Best You Could,’ Says Iron Man Ac...
2        New Emails Reveal Warm Relationship Between Ka...
3        Donald Trump Jr. Gets Slammed Over Racist Birt...
4            God Urges Rick Perry Not To Run For President
                               ...                        
12501                       Lapdogs of the Corporate Press
12502    Annoying Coworker Precedes All Nouns With 'Qui...
12503    Report: North Dakota Leads Nation In Parking A...
12504    The Election Night Drinking Game: HyperVocal E...
12505                        Conjoined Twin Hogging Kidney
Name: headlines, Length: 12506, dtype: object

In [11]:
# Every word map to unique number
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok,)
tokenizer.fit_on_texts(input_seq)
word_index = tokenizer.word_index

# Convert input text to numerical value
input_sentences = tokenizer.texts_to_sequences(input_seq)
# Create Every input sentence lenght same.
input_padded_sentences = pad_sequences(input_sentences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [12]:
# Target column all value convert them numerical value.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(target_seq)
label = le.transform(target_seq)
label

array([0, 1, 0, ..., 1, 0, 1])

In [13]:
# Split them training and validation data set for using sklearn train_test_split() function.
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(input_padded_sentences,label,test_size=0.2,
                                                 random_state=42)

len(x_train),len(x_test),len(y_train),len(y_test)

(10004, 2502, 10004, 2502)

In [14]:
# Create a Callback for using when I train model.
e_callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    restore_best_weights=True,
)
bm_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/',monitor='val_loss',save_best_only=True)


In [39]:
# Build Model Architecture
tf.keras.backend.clear_session()
np.random.seed(42)
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128,recurrent_dropout = 0.5 , dropout = 0.3, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,recurrent_dropout = 0.2 , dropout = 0.2,return_sequences=True)),
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [40]:
x_train.shape,y_train.shape

((10004, 100), (10004,))

In [41]:
# Train the model 
history = model.fit(x_train,tf.expand_dims(y_train,axis=1),
                    validation_data = (x_test,tf.expand_dims(y_test,axis=1)),
                    epochs=5,
                    batch_size=32,
                    callbacks=[e_callbacks,bm_callback])

Epoch 1/5
313/313 [==============================] - 534s 2s/step - loss: 0.5585 - accuracy: 0.6870 - val_loss: 0.3511 - val_accuracy: 0.8745
Epoch 2/5
313/313 [==============================] - 481s 2s/step - loss: 0.2350 - accuracy: 0.9164 - val_loss: 0.2520 - val_accuracy: 0.9046
Epoch 3/5
313/313 [==============================] - 462s 1s/step - loss: 0.1377 - accuracy: 0.9547 - val_loss: 0.2592 - val_accuracy: 0.9055
Epoch 4/5
313/313 [==============================] - 463s 1s/step - loss: 0.1006 - accuracy: 0.9690 - val_loss: 0.2947 - val_accuracy: 0.8995


In [42]:
model.evaluate(x_test,tf.expand_dims(y_test,axis=1))

79/79 [==============================] - 8s 105ms/step - loss: 0.2520 - accuracy: 0.9046


[0.2520469129085541, 0.9046002626419067]

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        112128    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        123648    
 nal)                                                            
                                                                 
 dense (Dense)               (None, None, 32)          4128      
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 dense_1 (Dense)             (None, None, 1)           3

In [47]:
model.save('/kaggle/working/sarcasm_detect.h5')

In [49]:
import tensorflow as tf
load_model  = tf.keras.models.load_model('/kaggle/working/sarcasm_detect.h5')
load_model.evaluate(x_test,tf.expand_dims(y_test,axis=1))

79/79 [==============================] - 10s 109ms/step - loss: 0.2520 - accuracy: 0.9046


[0.2520469129085541, 0.9046002626419067]

In [86]:
for i,input_text_with_label in enumerate(zip(x_test[50:],y_test[50:])):
    if i > 5:
        break
    else:
        test_input_seq = input_text_with_label[0]
        actual_label = input_text_with_label[1]
        
        probs = np.squeeze(load_model.predict(np.array([test_input_seq])))
        pred = int(np.round(np.max(probs)))
        print(f"Prediction Value Is: {pred}")
        print(f'Actual Value Is: {actual_label}')

1/1 [==============================] - 0s 135ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 [==============================] - 0s 127ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 [==============================] - 0s 133ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 [==============================] - 0s 131ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 [==============================] - 0s 125ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 [==============================] - 0s 117ms/step
Prediction Value Is: 1
Actual Value Is: 1
